In [1]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

# Load Run

In [2]:
import pandas as pd
# Load a specific run

runs = [
    "VAL_naive_doctaet-simple_zs-mistralaiMistral7BInstructv03-20240611-095123",
    "VAL_naive_doctaet-simple_fs-mistralaiMistral7BInstructv03-20240611-102154",
    "VAL_naive_doctaet-simple_zs-googlegemma7bit-20240611-130659",
    "VAL_naive_doctaet-simple_fs-googlegemma7bit-20240611-133609",
    "VAL_llama3_70b_fewshot_optimized02-20240531-105142",
    "VAL_llama3_8b_few_shot_template_initial-20240604-125136"
]

# Top runs on Test:
# - TEST_llama3_70b_fewshot_initial-20240601-211554
dfs = []

for run in runs:
    path = f"results/{run}/df.feather"

    df = pd.read_feather(path)
    try:
        df = df.rename({"annontation": "annotation"}, axis=1) # typo in early versions
    except:
        pass
    dfs.append(df)


# Post Processing

In [4]:
for df in dfs:
    df["annotation"] = df["annotation"].apply(format)
    df["annotation"] = df["annotation"].astype(pd.StringDtype())
    df["ground_truth"] = df["ground_truth"].astype(pd.StringDtype())

In [5]:
df.dtypes

run             object
f               object
annotation      string
ground_truth    string
dtype: object

# Evaluation

In [8]:
# evaluate
from scoring_program.evaluation import Metrics

results = []
for df in dfs:
    result = Metrics.evaluate_property_wise_json_based(
        label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"])
    )

    result.update(
        Metrics.evaluate_rouge(label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"]))
    )
    result["run"] = df["run"][0]
    result["score"] = result["partial_f1s_overall"]
    result["num_samples"] = len(df)
    results.append(pd.DataFrame([result]))

df_res = pd.concat(results)
df_res

,general_accuracy,exact_recalls_task,exact_recalls_dataset,exact_recalls_metric,exact_recalls_Score,exact_recalls_overall,partial_recalls_task,partial_recalls_dataset,partial_recalls_metric,partial_recalls_Score,...,partial_f1s_metric,partial_f1s_Score,partial_f1s_overall,rouge1,rouge2,rougeL,rougeLsum,run,score,num_samples
0,51.0,0.99,0.99,4.93,0.99,1.97,6.40,0.99,4.93,0.97,...,5.35,1.50,3.81,18.73,11.54,17.80,17.47,VAL_naive_doctaet-simple_zs-mistralaiMistral7B...,3.81,100
0,64.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,34.51,14.45,33.73,33.71,VAL_naive_doctaet-simple_fs-mistralaiMistral7B...,0.00,100
0,51.0,0.00,0.00,0.00,0.00,0.00,0.49,0.00,0.00,0.00,...,0.00,0.00,0.15,13.28,2.28,11.40,11.30,VAL_naive_doctaet-simple_zs-googlegemma7bit-20...,0.15,100
0,58.0,0.00,0.00,0.00,0.00,0.00,0.49,0.00,0.49,0.00,...,0.98,0.00,0.49,16.87,1.52,16.16,16.11,VAL_naive_doctaet-simple_fs-googlegemma7bit-20...,0.49,100
0,49.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,49.00,0.00,49.00,49.00,VAL_llama3_70b_fewshot_optimized02-20240531-10...,0.00,100
0,69.0,4.93,13.79,7.39,1.97,7.02,10.84,14.78,8.87,1.94,...,7.16,1.43,6.91,34.03,7.58,33.12,33.04,VAL_llama3_8b_few_shot_template_initial-202406...,6.91,100


In [7]:
list(df["annotation"])

["[{'LEADERBOARD': {'Task': 'Illumination of Monotone Orthogonal Polygons', 'Dataset': 'Monotone Orthogonal Polygons', 'Metric': '', 'Score': ''}}, {'LEADERBOARD': {'Task': 'Illumination of Monotone Polygons', 'Dataset': 'Monotone Polygons', 'Metric': '', 'Score': ''}}]",
 'unanswerable\n',
 "[{'LEADERBOARD': {'Task': 'Lecture Transcripts', 'Dataset': 'arXIV', 'Metric': 'ROUGE-2', 'Score': '16.71'}}, {'LEADERBOARD': {'Task': 'Lecture Transcripts', 'Dataset': 'arXiv', 'Metric': 'ROUGE-1', 'Score': '42.10'}}, {'LEADERBOARD': {'Task': 'Lecture Transcripts', 'Dataset': 'PubMed', 'Metric': 'ROUGE-2', 'Score': '23.28'}}, {'LEADERBOARD': {'Task': 'Lecture Transcripts', 'Dataset': 'PubMed', 'Metric': 'ROUGE-1', 'Score': '50.25'}}]",
 "[{'LEADERBOARD': {'Task': 'LLM Distractor', 'Dataset': 'Distractor', 'Metric': 'Average Test Accuracy', 'Score': ''}}, {'LEADERBOARD': {'Task': 'LLM Number of Digits', 'Dataset': 'Number of Digits', 'Metric': 'Average Test Accuracy', 'Score': ''}}, {'LEADERBOARD'